In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

ModuleNotFoundError: No module named 'pandas'

In [ ]:
stocks = pd.read_csv('s&p500_stocks.csv')
stock_symbols = stocks['Ticker'].tolist()
#stock_symbols

In [ ]:
IEX_CLOUD_API_TOKEN = 'sk_96206e7edc754fa6a6426b3e2a5adfcf'

In [ ]:
# Creating a pandas dataframe that will eventually be a our final excel sheet

my_columns = ['Ticker', 'Price', 'Market Capitalisation', 'Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [ ]:
# Batch Api Requests for all stocks in FTSE 100


# Create a string of the stocks to place in the URL
stock_symbols_string = ','.join(stock_symbols)
batch_url = f'https://api.iex.cloud/v1/data/CORE/QUOTE/{stock_symbols_string}?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_url).json()


for i in range(0, len(data)):
    new_row = pd.Series({
        'Ticker': data[i]['symbol'],
        'Price': data[i]['latestPrice'],
        'Market Capitalisation': data[i]['marketCap'],
        'Number of shares to buy':'n/a'
    })

    final_dataframe.loc[len(final_dataframe)] = new_row

#final_dataframe

In [ ]:

def portfolio_size_input():
    portfolio_size = input('Enter how much you would like to invest in dollars: ')

    try:
        val = float(portfolio_size)

        if val > 0:
            return val
        else: 
            print("please enter a valid number")
            portfolio_size_input()
    except ValueError:
        print("please enter a valid number")
        portfolio_size_input()
    
    return 


val = portfolio_size_input()
val


In [ ]:
position_size = val / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['Price'][i])

#final_dataframe

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [ ]:
background_color = '#FFFFFF'
font_color = '#000000'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.close()